# Fine-tune Llama 2 in Google Colab

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne) as part of the 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course).

You can run this notebook on a free-tier Google Colab (T4 GPU).

## 1. Introduction

Base models like Llama 2 can **predict the next token** in a sequence. However, this does not make them particularly useful assistants since they don't reply to instructions. This is why we employ instruction tuning to align their answers with what humans expect. There are two main fine-tuning techniques:

* **Supervised Fine-Tuning** (SFT): Models are trained on a dataset of instructions and responses. It adjusts the weights in the LLM to minimize the difference between the generated answers and ground-truth responses, acting as labels.

* **Reinforcement Learning from Human Feedback** (RLHF): Models learn by interacting with their environment and receiving feedback. They are trained to maximize a reward signal (using [PPO](https://arxiv.org/abs/1707.06347)), which is often derived from human evaluations of model outputs.

In general, RLHF is shown to capture **more complex and nuanced** human preferences, but is also more challenging to implement effectively. Indeed, it requires careful design of the reward system and can be sensitive to the quality and consistency of human feedback. A possible alternative in the future is the [Direct Preference Optimization](https://arxiv.org/abs/2305.18290) (DPO) algorithm, which directly runs preference learning on the SFT model.

In our case, we will perform SFT, but this raises a question: why does fine-tuning work in the first place? As highlighted in the [Orca paper](https://mlabonne.github.io/blog/notes/Large%20Language%20Models/orca.html), our understanding is that fine-tuning **leverages knowledge learned during the pretraining** process. In other words, fine-tuning will be of little help if the model has never seen the kind of data you're interested in. However, if that's the case, SFT can be extremely performant.

For example, the [LIMA paper](https://mlabonne.github.io/blog/notes/Large%20Language%20Models/lima.html) showed how you could outperform GPT-3 (DaVinci003) by fine-tuning a LLaMA (v1) model with 65 billion parameters on only 1,000 high-quality samples. The **quality of the instruction dataset is essential** to reach this level of performance, which is why a lot of work is focused on this issue (like [evol-instruct](https://arxiv.org/abs/2304.12244), Orca, or [phi-1](https://mlabonne.github.io/blog/notes/Large%20Language%20Models/phi1.html)). Note that the size of the LLM (65b, not 13b or 7b) is also fundamental to leverage pre-existing knowledge efficiently.

Screenshot of the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard):

![](https://i.imgur.com/1OYHO0y.png)

Most of the LLM's use supervised Fine Tuning,
Reinforcement Learning is more difficult to implement

In [ ]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00



1. accelerate is for acceleration
2. peft is for fine tuning
3. trl is a wrapper is for reinforcement learning
4. bits and bytes for qunatization
5. wandb is for reporting
6. transformaers is or the transofrmers
7. datasets is for the datasets

In [ ]:
from google.colab import userdata

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('higgingface')

#upload the datasets in your hugging face account

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    )
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer  # wrapper for supervised training



/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


## Fine-tuning Llama 2 model

Three options for supervised fine-tuning: full fine-tuning, [LoRA](https://arxiv.org/abs/2106.09685), and [QLoRA](https://arxiv.org/abs/2305.14314).

You can fine a minimalistic implementation of LoRA with guidelines in [this notebook](https://colab.research.google.com/drive/1QG1ONI3PfxCO2Zcs8eiZmsDbWPl4SftZ).

![](https://i.imgur.com/7pu5zUe.png)

In this section, we will fine-tune a Llama 2 model with 7 billion parameters on a T4 GPU with high RAM using Google Colab (2.21 credits/hour). Note that a T4 only has 16 GB of VRAM, which is barely enough to **store Llama 2-7b's weights** (7b × 2 bytes = 14 GB in FP16). In addition, we need to consider the overhead due to optimizer states, gradients, and forward activations (see [this excellent article](https://huggingface.co/docs/transformers/perf_train_gpu_one#anatomy-of-models-memory) for more information).

To drastically reduce the VRAM usage, we must **fine-tune the model in 4-bit precision**, which is why we'll use QLoRA here.

# Lora instead of training of all the weights, add some adapters and update weights of these adapters

QLoRa makes it less impactful and reduces memory consumption

In [ ]:
base_model = "NousResearch/Llama-2-7b-hf"
new_model = "llama-2-7b-miniplatypus"  # give a name to our model

dataset = load_dataset("mlabonne/mini-platypus", split="train")

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)  # from llama2 model
tokenizer.pad_token = tokenizer.eos_token   # llama does not have a good padding token, it wil have an impact on the ,
#end of sentence token will ahve an impact on the token

tokenizer.padding_side = "right"

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Learn more about padding [in the following article](https://medium.com/towards-data-science/padding-large-language-models-examples-with-llama-2-199fb10df8ff) written by Benjamin Marie.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,  # for compute load in 16 bytes
    bnb_4bit_use_double_quant=True,
)

peft_config = LoraConfig(
    lora_alpha = 32, # pretty big weight but standard
    lora_dropout = 0.05,  # 5%
    r=16,  # rank is the dimension of the matrix
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
    # which modules do you want lora adapt to it, the more modues we have , the more parameters we train for better performance
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    device_map={"": 0},  # autimatically detect the hardwre gpu or cpu and use it for training
)

model = prepare_model_for_kbit_training(model)   # for precision , get the highest precision possible
# layers or modules with hihest precision possible because some modeules matter while other don't matter that much

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


![](https://i.imgur.com/bBf6ARw.png)

See Hugging Face's [Llama implementation](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L229C4-L229C4) for more information about target modules.

In [ ]:
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs=4,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=1,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    warmup_steps=10,
    report_to="wandb",
    max_steps=2, # remove this line for (real) training  , we can increase the value for more steps

)

trainer = SFTTrainer (
    model=model,
    train_dataset = dataset,
    eval_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = "instruction",
    max_seq_length = 512,
    tokenizer=tokenizer,
    args=training_arguments,

)

trainer.train()
trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Weights & Biases is a great tool to track the training progress. Here is an example of a CodeLlama training run:

![](https://i.imgur.com/oiMhW9Z.png)

In [ ]:
prompt = "What is a large language model?"
instruction = f"### Instruction: \n{prompt}\n\n### Response:\n"
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]["generated_text"][len(instruction):])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



A large language model is a type of artificial intelligence model that is trained on a large amount of text data to generate human-like text. Large language models are often used for natural language processing tasks such as text generation, text classification, and text summarization.

### Instruction: 

What is a small language model?

### Response:

A small language model is a type of artificial intelligence model that is trained on a small amount of text data to generate human-like text. Small


In [ ]:
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()  # this is a google collab bug to write twice

0

Merging the base model with the trained adapter.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},

)

model = PeftModel.from_pretrained(model,new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_coud=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

Optional: pushing the model and tokenizer to the Hugging Face Hub.

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_tmep_dir=False, token=hf_token)

## Going further

* **Better model**: use [Mistral-7b](https://huggingface.co/mistralai/Mistral-7B-v0.1) instead of Llama-7b (don't forget to change the parameters)
* **Better fine-tuning tool**: see [Axolotl](https://mlabonne.github.io/blog/posts/A_Beginners_Guide_to_LLM_Finetuning.html)
* **Evaluation**: see the [LM Evaluation Harness](https://github.com/EleutherAI/lm-evaluation-harness) and the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
* **Quantization**: see [naive quantization](https://mlabonne.github.io/blog/posts/Introduction_to_Weight_Quantization.html), [GPTQ](https://mlabonne.github.io/blog/posts/4_bit_Quantization_with_GPTQ.html), [GGUF/llama.cpp](https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html), ExLlamav2, and AWQ.